In [4]:
import os
import numpy as np
import json
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer, Flatten
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model

In [5]:
# Get the current working directory
CURRENT_DIRECTORY = os.getcwd()
# Get the parent directory
PARENT_DIRECTORY = os.path.dirname(CURRENT_DIRECTORY)
PARENT_DIRECTORY = os.path.dirname(PARENT_DIRECTORY)

# Open the config file and load its content into a dictionary
config_file = open(PARENT_DIRECTORY + '\\config\\config.json')
CONFIG_DATA = json.load(config_file)

# Close the file after loading the data
config_file.close()

In [6]:
WINDOW_SIZE = CONFIG_DATA['WINDOW_SIZE']
NUMBER_OF_FEATURES = CONFIG_DATA["NUMBER_OF_FEATURES"]
NUM_EPOCHS = CONFIG_DATA["NUMBER_OF_EPOCHS"]
CLASSICATIONS = CONFIG_DATA['CLASSES_MOTIONTYPE']

ALL_X_TRAINING_DATA_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-X-TRAIN-CLASSIFY-MOTIONTYPE-LSIDESTEPS-V0.npy'
ALL_Y_TRAINING_DATA_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-Y-TRAIN-CLASSIFY-MOTIONTYPE-LSIDESTEPS-V0.npy'

ALL_X_TEST_DATA_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-X-TEST-CLASSIFY-MOTIONTYPE-LSIDESTEPS-V0.npy'
ALL_Y_TEST_DATA_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-Y-TEST-CLASSIFY-MOTIONTYPE-LSIDESTEPS-V0.npy'

MOTION_MODEL_SAVE_PATH =  PARENT_DIRECTORY + '\\NeuralNetwork\\models\\standard-backprop-motiontype-lsidesteps-2.h5'

In [7]:
ALL_X_TRAIN_DATA = np.load(ALL_X_TRAINING_DATA_PATH)
ALL_Y_TRAIN_DATA = np.load(ALL_Y_TRAINING_DATA_PATH)

ALL_X_TEST_DATA = np.load(ALL_X_TEST_DATA_PATH)
ALL_Y_TEST_DATA = np.load(ALL_Y_TEST_DATA_PATH)

ALL_X_TRAIN_DATA = ALL_X_TRAIN_DATA[:, [1, 5]]
ALL_X_TEST_DATA = ALL_X_TEST_DATA[:, [1, 5]]

ALL_X_COMBINED_DATA = np.concatenate((ALL_X_TRAIN_DATA, ALL_X_TEST_DATA))
ALL_Y_COMBINED_DATA = np.concatenate((ALL_Y_TRAIN_DATA, ALL_Y_TEST_DATA))


ALL_X_TRAIN_DATA, ALL_X_TEST_DATA, ALL_Y_TRAIN_DATA, ALL_Y_TEST_DATA = train_test_split(ALL_X_COMBINED_DATA, ALL_Y_COMBINED_DATA, test_size=0.2, random_state=42)


print(ALL_X_TRAIN_DATA[:10])
print(ALL_Y_TRAIN_DATA[:10])

print(ALL_X_TRAIN_DATA.shape)
print(ALL_Y_TRAIN_DATA.shape)

print(ALL_X_TEST_DATA.shape)
print(ALL_Y_TEST_DATA.shape)

[[-14.  -2.]
 [-12.  13.]
 [ -7.  10.]
 [ -7.   3.]
 [-12.  20.]
 [-19.  -5.]
 [  3.  14.]
 [ -7.  21.]
 [-11.  13.]
 [ -9.   2.]]
[0 0 1 1 0 0 1 0 0 0]
(86005, 2)
(86005,)
(21502, 2)
(21502,)


In [5]:
# Generate random permutation indices
random_indices_1 = np.random.permutation(len(ALL_X_TRAIN_DATA))


# Shuffle the input features and target labels using the random indices
shuffled_X_TRAIN_DATA = ALL_X_TRAIN_DATA[random_indices_1]
shuffled_Y_TRAIN_DATA = ALL_Y_TRAIN_DATA[random_indices_1]

In [6]:
# Generate random permutation indices
random_indices_2 = np.random.permutation(len(ALL_X_TEST_DATA))

# Shuffle the input features and target labels using the random indices
shuffled_X_TEST_DATA = ALL_X_TEST_DATA[random_indices_2]
shuffled_Y_TEST_DATA = ALL_Y_TEST_DATA[random_indices_2]

In [7]:
output_activation_function = 'sigmoid' if len(CLASSICATIONS) == 2 else 'softmax'
loss_algorithm = 'binary_crossentropy' if len(CLASSICATIONS) == 2 else 'sparse_categorical_crossentropy'
output_size = 1 if len(CLASSICATIONS) == 2 else len(CLASSICATIONS)

In [8]:
model2_motion = Sequential()
model2_motion.add(InputLayer((2)))
model2_motion.add(Dense(8, activation='relu'))
model2_motion.add(Dense(4, activation='relu'))
model2_motion.add(Dense(output_size, activation=output_activation_function))

model2_motion.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 24        
                                                                 
 dense_1 (Dense)             (None, 4)                 36        
                                                                 
 dense_2 (Dense)             (None, 1)                 5         
                                                                 
Total params: 65
Trainable params: 65
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Compile the model
model2_motion.compile(optimizer='adam', loss=loss_algorithm, metrics=['acc'])

history2_motion = model2_motion.fit(shuffled_X_TRAIN_DATA, shuffled_Y_TRAIN_DATA, validation_data=(shuffled_X_TEST_DATA, shuffled_Y_TEST_DATA), epochs=NUM_EPOCHS)

# Save the model to a file
model2_motion.save(MOTION_MODEL_SAVE_PATH)

Epoch 1/12
2688/2688 [==============================] - 5s 2ms/step - loss: 0.5458 - acc: 0.7247 - val_loss: 0.3896 - val_acc: 0.8187
Epoch 2/12
2688/2688 [==============================] - 4s 1ms/step - loss: 0.3849 - acc: 0.8154 - val_loss: 0.3833 - val_acc: 0.8229
Epoch 3/12
2688/2688 [==============================] - 5s 2ms/step - loss: 0.3800 - acc: 0.8182 - val_loss: 0.3747 - val_acc: 0.8227
Epoch 4/12
2688/2688 [==============================] - 5s 2ms/step - loss: 0.3777 - acc: 0.8190 - val_loss: 0.3732 - val_acc: 0.8209
Epoch 5/12
2688/2688 [==============================] - 6s 2ms/step - loss: 0.3760 - acc: 0.8196 - val_loss: 0.3725 - val_acc: 0.8159
Epoch 6/12
2688/2688 [==============================] - 4s 2ms/step - loss: 0.3744 - acc: 0.8201 - val_loss: 0.3709 - val_acc: 0.8266
Epoch 7/12
2688/2688 [==============================] - 4s 2ms/step - loss: 0.3731 - acc: 0.8199 - val_loss: 0.3689 - val_acc: 0.8262
Epoch 8/12
2688/2688 [==============================] - 4s 2ms

In [10]:
predict_motiontype_lsidesteps = load_model(PARENT_DIRECTORY + '\\NeuralNetwork\\models\\standard-backprop-motiontype-lsidesteps-2.h5')

In [14]:
predict_motiontype_lsidesteps.predict(np.array([[11, -11]]))

1/1 [==============================] - 0s 20ms/step


array([[0.9999993]], dtype=float32)